#### Importing Dependencies

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings( "ignore" )

#### Data Collection and Analysis

In [2]:
# storing the csv file as pandas dataframe for better visualization
credit_card_data = pd.read_csv("./credit_card.csv")

In [3]:
# displaying the first 5 rows
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# getting the shape of credit card data
print(f"(rows, columns): {credit_card_data.shape}")

(rows, columns): (284807, 31)


In [5]:
# getting some more information about credit card dataset
credit_card_data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
# checking for missing values
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
# distribution in legal transaction and fraud transaction
credit_card_data['Class'].value_counts()

# the data is unbalanced
# 0 -> legal transaction
# 1 -> fraud transaction

0    284315
1       492
Name: Class, dtype: int64

#### Data Preprocessing

In [8]:
# seperate the legal and fraud transaction list
legal_transaction = credit_card_data[credit_card_data.Class == 0]
fraud_transaction = credit_card_data[credit_card_data.Class == 1]

In [9]:
print("legal transaction", legal_transaction.shape)
print("fraud transaction", fraud_transaction.shape)

legal transaction (284315, 31)
fraud transaction (492, 31)


In [10]:
# statistical description of Amount column in whole dataset
credit_card_data.Amount.describe()

count    284807.000000
mean         88.349619
std         250.120109
min           0.000000
25%           5.600000
50%          22.000000
75%          77.165000
max       25691.160000
Name: Amount, dtype: float64

In [11]:
# statistical description of Amount column in legal transaction dataset
legal_transaction.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [12]:
# statistical description of Amount column in fraud transaction dataset
fraud_transaction.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [13]:
# group data by Class and compare the means of all features
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


##### Perform Under-Sampling
Since the data is unbalanced, we have to perform Under-Sampling, i.e. build a sample dataset containing similar distribution of legal transaction and fraud transaction

In [14]:
# since number of fraud transaction in dataset are 492, we will choose 492 samples from legal transaction dataset
legal_transaction_sample = legal_transaction.sample(n=492)

In [15]:
# concatenating two dataframes
preprocessed_credit_card_data = pd.concat([legal_transaction_sample, fraud_transaction], axis=0)

In [16]:
# displaying first 5 rows of preprocessed credit card data
preprocessed_credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
281645,170328.0,0.669369,0.802239,-0.548859,-0.461751,1.103532,-0.877397,1.152531,-0.446391,0.036620,...,-0.294856,-0.573588,0.208144,0.651524,-0.988035,-0.034410,-0.142753,-0.182496,1.98,0
117838,74837.0,1.277773,0.721936,-0.547931,0.865114,0.190828,-1.437579,0.471852,-0.369401,-0.303554,...,-0.143526,-0.311912,-0.094104,0.298321,0.601125,0.365853,-0.024796,0.044816,0.76,0
8210,11052.0,1.283984,-0.053111,0.634714,0.204341,-0.617864,-0.539234,-0.485629,-0.136396,1.856679,...,-0.108205,-0.075438,-0.074796,-0.138372,0.260158,1.065848,-0.093784,-0.000703,15.95,0
133884,80561.0,0.894258,-0.327235,-0.166174,1.135400,-0.030057,0.119024,0.240464,0.094359,0.039812,...,0.099504,0.059479,-0.289118,-0.303268,0.692327,-0.256227,-0.012894,0.020037,151.51,0
237581,149306.0,0.596947,-0.596088,0.951526,-1.612854,-1.838533,0.199712,-2.371400,-2.475910,0.025477,...,-1.473572,0.282703,0.205624,0.009291,0.099413,1.239882,-0.093300,0.040821,27.90,0


In [17]:
# displaying last 5 rows of preprocessed credit card data
preprocessed_credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [18]:
preprocessed_credit_card_data["Class"].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [19]:
# group the preprocessed data by Class and compare the means
preprocessed_credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,89547.910569,-0.029912,-0.050275,0.203146,0.058915,0.147324,0.044334,0.043606,0.036962,-0.028734,...,0.045335,-0.033771,0.010387,-0.015819,0.06947,0.007639,0.022569,0.003544,-0.013919,86.908862
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.10513,0.041449,0.051648,0.170575,0.075667,122.211321


#### Splitting Data into Features and Targets

In [20]:
# Features are all the columns except Class and the Target is the Class column
X = preprocessed_credit_card_data.drop(columns='Class', axis=1)
Y = preprocessed_credit_card_data['Class']

In [21]:
X

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
281645,170328.0,0.669369,0.802239,-0.548859,-0.461751,1.103532,-0.877397,1.152531,-0.446391,0.036620,...,-0.052753,-0.294856,-0.573588,0.208144,0.651524,-0.988035,-0.034410,-0.142753,-0.182496,1.98
117838,74837.0,1.277773,0.721936,-0.547931,0.865114,0.190828,-1.437579,0.471852,-0.369401,-0.303554,...,-0.012587,-0.143526,-0.311912,-0.094104,0.298321,0.601125,0.365853,-0.024796,0.044816,0.76
8210,11052.0,1.283984,-0.053111,0.634714,0.204341,-0.617864,-0.539234,-0.485629,-0.136396,1.856679,...,-0.098595,-0.108205,-0.075438,-0.074796,-0.138372,0.260158,1.065848,-0.093784,-0.000703,15.95
133884,80561.0,0.894258,-0.327235,-0.166174,1.135400,-0.030057,0.119024,0.240464,0.094359,0.039812,...,0.049400,0.099504,0.059479,-0.289118,-0.303268,0.692327,-0.256227,-0.012894,0.020037,151.51
237581,149306.0,0.596947,-0.596088,0.951526,-1.612854,-1.838533,0.199712,-2.371400,-2.475910,0.025477,...,0.104137,-1.473572,0.282703,0.205624,0.009291,0.099413,1.239882,-0.093300,0.040821,27.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,1.252967,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.226138,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.247968,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.306271,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00


In [22]:
Y

281645    0
117838    0
8210      0
133884    0
237581    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64

#### Splitting preprocessed data into test and train data from Scratch

In [23]:
def test_train_split(dataset, target, test_size=0.25):
    train_data = list()
    test_data = dataset.copy()
    train_size = round((1-test_size)*len(dataset))
    
    while len(train_data) < train_size:
        i = rd.randrange(len(test_data)) # finding a random index from test_data which is initially a copy of dataset
        train_data.append(test_data.iloc[i, :]) # appending the row at random index to train data
        test_data.drop(test_data.index[i], inplace=True) # removing the row at random index from test data
    
    train_data = pd.DataFrame(train_data) 
    
    X_train_data = train_data.drop(columns=target, axis=1)
    Y_train_data = train_data[target]
    X_test_data = test_data.drop(columns=target, axis=1)
    Y_test_data = test_data[target]
    
    return X_train_data, Y_train_data, X_test_data, Y_test_data


In [24]:
# splitting the preprocessed data
X_train, Y_train, X_test, Y_test = test_train_split(preprocessed_credit_card_data, target="Class", test_size=0.2)

#### Logistic Regression Model from Scratch

Sigmoid function <br><br>
$
g(z) = \frac{1}{1+e^{-z}}
$
<br><br>
$
\hat{y} = \frac{1}{1+e^{-(wX+b)}}
$
<br><br>
Cost function <br><br>
$
J(w,b) = -\frac{1}{m} \sum_{i=1}^{m} [(y^{(i)}\log{\hat{y}^{(i)}})+(1-y^{(i)})\log{(1-\hat{y}^{(i)})}]
$
<br><br>
Gradiants
<br><br>
$
dw = -\frac{1}{m} \sum_{i=1}^{m} [x^{(i)}(y^{(i)} - \hat{y}^{(i)})]
$
<br><br>
$
db = -\frac{1}{m} \sum_{i=1}^{m} [(y^{(i)} - \hat{y}^{(i)})]
$
<br><br>
Normalizing equation
<br><br>
$
z = \frac{(x-\mu)}{\sigma}
$

In [33]:
# Logistic Regression model
class LogisticRegressionScratch():
    
    # utility function for defining sigmoid function
    def _sigmoid(self, z):
        g = 1/(1+np.exp(-z))
        return g
    
    # utility function for finding the cost/loss function J(w, b)
    def _cost(self, Y, Y_cap):
        J = -np.mean(Y*np.log(Y_cap) + (1-Y)*np.log(1-Y_cap))
        return J
    
    # utility function for finding the gradiants dw, db
    # dw is the partial derivative of cost function J(w, b) with respect to w
    # db is the partial derivative of cost function J(w, b) with respect to b
    def _gradiants(self, Y, Y_cap, X):
        m = X.shape[0]
        # print(m)
        dw = (1/m)*np.dot(X.T, (Y_cap-Y))
        db = (1/m)*np.sum((Y_cap-Y))
        return dw, db
    
    # utility function for normalizing inputs
    def _normalize(self, X):
        n = X.shape[1]
        for i in range(n):
            X = (X-X.mean(axis=0))/X.std(axis=0)
        return X
    
    # train function
    # alpha is the learning rate
    def train(self, X, Y, batch_size, alpha, iters=1000):
        m, n = X.shape
        w = np.zeros((n, 1))
        b = 0
        
        Y = Y.values.reshape(m, 1)
        x = self._normalize(X)
        costs = []
        
        for iter in range(iters):
            for i in range(((m-1)//batch_size)+1):
                s = i*batch_size
                e = s + batch_size
                xb = x[s:e]
                yb = Y[s:e]
                
                y_cap = self._sigmoid(np.dot(xb, w) + b)
                dw, db = self._gradiants(yb, y_cap, xb)
                
                # updating w and b for each iteration to minimize the cost function
                w -= alpha*dw
                b -= alpha*db
            
            cost = self._cost(Y, self._sigmoid(np.dot(x, w)+b))
            costs.append(cost)
        # print(len(w))
        return w, b, costs
    
    # prediction function
    def predict(self, X):
        x = self._normalize(X)
        y_cap = self._sigmoid(np.dot(x, w)+b)
        predict_list = [1 if i>=0.5 else 0 for i in y_cap]
        return np.array(predict_list)
    
model = LogisticRegressionScratch()
w, b, l = model.train(X_train, Y_train, batch_size=10, alpha=0.01, iters=1000)

In [35]:
# function for checking the accuracy of the model
def accuracy_score(y, y_hat):
    accuracy = np.sum(y == y_hat) / len(y)
    return accuracy

accuracy_score(Y_test, y_hat=model.predict(X_test))

0.934010152284264

#### Splitting preprocessed data into test and train data using Scikit-Learn

In [61]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [62]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


#### Logistic Regression Model using Scikit-Learn

In [63]:
model2 = LogisticRegression()

In [64]:
# training the data with the model
model2.fit(X_train, Y_train)

LogisticRegression()

In [65]:
# accuracy on training data
X_train_prediction = model2.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [66]:
print("Accuracy of Training data: ", training_data_accuracy)

Accuracy of Training data:  0.9390088945362135


In [67]:
# accuracy on testing data
X_test_prediction = model2.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [68]:
print("Accuracy of Testing data: ", testing_data_accuracy)

Accuracy of Testing data:  0.9441624365482234
